In [1]:
import pickle
infile = './exports/hbs_result.pt'
with open(infile,'rb') as f:
    data = pickle.load(f)
    print(data)

{'data': {'company': {'train': <torch.utils.data.dataloader.DataLoader object at 0x000001D4F36E2EB0>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x000001D4F38A6EE0>}, 'brand': {'train': <torch.utils.data.dataloader.DataLoader object at 0x000001D49B82B580>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x000001D49B837100>}, 'category': {'train': <torch.utils.data.dataloader.DataLoader object at 0x000001D49B837AC0>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x000001D49B84CC70>}}, 'train_labels': <torch.utils.data.dataloader.DataLoader object at 0x000001D49B84CFD0>, 'test_labels': <torch.utils.data.dataloader.DataLoader object at 0x000001D49B85A100>, 'batch_size': 64}


In [19]:
data

{'data': {'company': {'train': <torch.utils.data.dataloader.DataLoader at 0x1ae9a57fd60>,
   'test': <torch.utils.data.dataloader.DataLoader at 0x1ae9a48bf40>},
  'brand': {'train': <torch.utils.data.dataloader.DataLoader at 0x1ae9a5d0be0>,
   'test': <torch.utils.data.dataloader.DataLoader at 0x1ae9a5d01c0>},
  'category': {'train': <torch.utils.data.dataloader.DataLoader at 0x1ae9a5d0e50>,
   'test': <torch.utils.data.dataloader.DataLoader at 0x1ae9a5d0df0>}},
 'train_labels': <torch.utils.data.dataloader.DataLoader at 0x1ae9a5d0820>,
 'test_labels': <torch.utils.data.dataloader.DataLoader at 0x1ae9a5d03a0>,
 'batch_size': 64}

In [20]:
trainloader =data['data']['company']['train']

In [25]:
iter(trainloader)

In [28]:
next(enumerate(trainloader))

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object

In [11]:
trainiter  = iter(trainloader)
next(trainiter)

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object

In [12]:
trainiter

In [28]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,Dataset,random_split
import torch
import pandas as pd
import numpy as np 
from pickle import dump
import json
from io import BytesIO
import os 
import zipfile
from zipfile import ZipFile
import requests

class ShoppersDataset(Dataset):
    """
        The class used to represent a given client's data set in VFL (no labels).

        Attributes
        __________
        X: numpy.ndarray
            A numppy array representing client's dataset. Each row represents a consumer's features.
        """
    def __init__(self,X):
        self.X = X
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index,:]
    
def create_train_test(df, train_index, test_index, batch_size, cols=None):
    """
    Create train and test DataLoader objects using specified parameters.

    Args:
        df: pandas.DataFrame
            DataFrame object containing data to be used for training and testing purposes.
        cols: list
            A subset of columns of `df` to include when creating train/test DataLoaders.
        train_index: array-like
            An iterable that contains indices representing training samples in `df`. 
        test_index: array-like
            An iterable that contains indices representing testing samples in `df`. 
        batch_size: int
            Batch size to use while training/testing models.
    
    Returns:
        tuple(DataLoader)
            A tuple that includes the train and test DataLoaders. 
    """
    torch.manual_seed(seed=0)
    
    if cols != None:
        train_ds = ShoppersDataset(
            df.loc[train_index][cols].copy().to_numpy()
        )
        test_ds = ShoppersDataset(
            df.loc[test_index][cols].copy().to_numpy()
        )
        train_dl = DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=False)
        test_dl = DataLoader(dataset=test_ds, batch_size=batch_size, shuffle=False)
        return train_dl,test_dl
    else:
        train_ds = ShoppersDataset(
            df.loc[train_index].copy().to_numpy()
        )
        test_ds = ShoppersDataset(
            df.loc[test_index].copy().to_numpy()
        )
        train_dl = DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=False)
        test_dl = DataLoader(dataset=test_ds, batch_size=batch_size, shuffle=False)
        return train_dl,test_dl

In [17]:
cols = None
if cols != None:
    print('not none')
else:
    print('none')

none


In [29]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,Dataset,random_split
import torch
import pandas as pd
import numpy as np 
from pickle import dump
import json
from io import BytesIO
import os 
import zipfile
from zipfile import ZipFile
import requests

data_filename = []
data=[] 
with ZipFile('./data/hbs_dataset.zip', 'r') as zipObj:
    listOfFileNames = zipObj.namelist()
    for fileName in listOfFileNames:
        if fileName.endswith('csv'): 
            # print(fileName)
            zipRead = zipObj.read(fileName)
            curr_df = pd.read_csv(BytesIO(zipRead))
            data.append(curr_df)
            data_filename.append(fileName)  

In [30]:
data[0].iloc[:,1:]

,C>G,C[C>G]G,C[C>G],[C>G]G,T>C,C[T>C]G,C[T>C],[T>C]G,T[T>C]A,T[T>C],...,G[T>G]G,G[T>G]A,T[T>G]C,C[C>A]A,C[T>G]T,A[T>A]G,A[T>G]A,C[T>A]A,A[T>A]A,target
0,0.167778,0.572886,-0.202459,0.979088,0.278607,0.551523,0.872449,0.146939,-0.388435,-0.193196,...,-0.939246,0.666221,-1.013557,0.744800,-0.459210,-1.263353,-1.532208,-0.633714,0.149173,GGG
1,0.554365,1.286023,0.911996,0.443742,-0.090261,-1.552856,-0.951789,-1.003374,-0.140761,0.133333,...,0.483854,-0.883130,1.458533,-0.173843,-0.459210,-0.646080,1.014398,0.277250,-0.934525,GGG
2,0.013144,2.355728,1.469224,0.086845,0.043873,-0.851397,0.177501,-0.428218,0.106912,-0.193196,...,-1.295021,1.182672,-0.024721,-1.092486,0.798900,0.588467,-0.004244,1.188213,-0.212060,GGG
3,1.095586,-1.209955,0.726254,-0.270052,0.345673,0.726888,1.393660,-0.181722,-0.883782,-0.955097,...,-0.583471,0.666221,-0.519139,0.744800,-1.717321,1.205741,1.523719,0.277250,-0.934525,GGG
4,0.322413,-0.140250,-0.202459,0.265294,0.412740,-0.500667,0.959318,0.064774,-0.388435,0.786390,...,0.128079,0.149771,0.964115,1.663443,2.057011,-0.646080,-0.513566,0.277250,-0.934525,GGG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,-1.997105,-1.209955,-1.316915,-1.519193,-1.465130,-0.149937,-1.559869,-1.332035,-0.140761,-0.846254,...,-0.227696,-1.916032,-0.024721,0.744800,-0.459210,0.588467,-0.004244,1.188213,-0.573292,BBB
596,-1.610519,-1.566523,-0.573945,-1.340744,-1.062729,-1.552856,-1.125526,-1.167705,-0.140761,0.242176,...,1.195404,2.215573,0.469697,0.744800,-1.088266,-0.028806,-0.513566,0.277250,-0.934525,BBB
597,0.554365,-0.853387,-1.316915,0.979088,-1.062729,-1.377491,-0.778052,-0.346052,-0.883782,-0.737411,...,0.483854,-0.366680,-1.507974,0.744800,0.798900,-1.263353,0.505077,0.277250,-0.934525,BBB
598,-1.146615,0.572886,-0.945430,-1.340744,-1.096263,-1.552856,-1.646737,-1.167705,1.345280,-0.193196,...,-0.939246,-0.883130,-0.024721,0.744800,-1.088266,-0.646080,-0.004244,-1.544677,-0.212060,BBB


In [31]:
target_code = {'RRR':0,'GGG':1,'BBB':2}
data[0]['target'] = data[0]['target'].map(target_code)
data[0]

,Unnamed: 0,C>G,C[C>G]G,C[C>G],[C>G]G,T>C,C[T>C]G,C[T>C],[T>C]G,T[T>C]A,...,G[T>G]G,G[T>G]A,T[T>G]C,C[C>A]A,C[T>G]T,A[T>A]G,A[T>G]A,C[T>A]A,A[T>A]A,target
0,GGG-0001,0.167778,0.572886,-0.202459,0.979088,0.278607,0.551523,0.872449,0.146939,-0.388435,...,-0.939246,0.666221,-1.013557,0.744800,-0.459210,-1.263353,-1.532208,-0.633714,0.149173,1
1,GGG-0002,0.554365,1.286023,0.911996,0.443742,-0.090261,-1.552856,-0.951789,-1.003374,-0.140761,...,0.483854,-0.883130,1.458533,-0.173843,-0.459210,-0.646080,1.014398,0.277250,-0.934525,1
2,GGG-0003,0.013144,2.355728,1.469224,0.086845,0.043873,-0.851397,0.177501,-0.428218,0.106912,...,-1.295021,1.182672,-0.024721,-1.092486,0.798900,0.588467,-0.004244,1.188213,-0.212060,1
3,GGG-0005,1.095586,-1.209955,0.726254,-0.270052,0.345673,0.726888,1.393660,-0.181722,-0.883782,...,-0.583471,0.666221,-0.519139,0.744800,-1.717321,1.205741,1.523719,0.277250,-0.934525,1
4,GGG-0006,0.322413,-0.140250,-0.202459,0.265294,0.412740,-0.500667,0.959318,0.064774,-0.388435,...,0.128079,0.149771,0.964115,1.663443,2.057011,-0.646080,-0.513566,0.277250,-0.934525,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,BBB-0110,-1.997105,-1.209955,-1.316915,-1.519193,-1.465130,-0.149937,-1.559869,-1.332035,-0.140761,...,-0.227696,-1.916032,-0.024721,0.744800,-0.459210,0.588467,-0.004244,1.188213,-0.573292,2
596,BBB-0126,-1.610519,-1.566523,-0.573945,-1.340744,-1.062729,-1.552856,-1.125526,-1.167705,-0.140761,...,1.195404,2.215573,0.469697,0.744800,-1.088266,-0.028806,-0.513566,0.277250,-0.934525,2
597,BBB-0162,0.554365,-0.853387,-1.316915,0.979088,-1.062729,-1.377491,-0.778052,-0.346052,-0.883782,...,0.483854,-0.366680,-1.507974,0.744800,0.798900,-1.263353,0.505077,0.277250,-0.934525,2
598,BBB-0199,-1.146615,0.572886,-0.945430,-1.340744,-1.096263,-1.552856,-1.646737,-1.167705,1.345280,...,-0.939246,-0.883130,-0.024721,0.744800,-1.088266,-0.646080,-0.004244,-1.544677,-0.212060,2


In [32]:
train_index, test_index = train_test_split(data[0].index.values,test_size=0.2,stratify=data[0]['target'])

In [33]:
train_index, test_index

(array([ 86, 228, 460, 239,  55, 371, 331, 336,  44,  91, 235, 334,   5,
        299, 461, 318,  33,  30, 113,  39, 314, 475, 309,  46, 528, 308,
        117, 203, 253, 348, 595, 270, 373, 511, 266, 456, 260, 397, 134,
        581, 245, 332, 558, 345, 473, 405,  89, 554, 139, 391, 280, 197,
        105, 438, 455, 223, 279, 257,   3, 307, 111, 311, 130, 416, 453,
        495, 118, 119, 286, 171, 501,  96, 285, 133, 363,  37,  50, 163,
        327, 497, 249, 557, 269, 537, 292, 375, 244, 585, 417, 432,  71,
        316, 400, 498, 578, 563, 593, 446, 582, 440, 381, 553,  14, 150,
        386, 448,  22, 403, 457, 503, 229, 420, 571, 162, 272, 384, 409,
        205, 521, 198,  13, 160, 243, 580, 298, 182, 520, 362, 462, 161,
        143, 341, 212,   7, 479,  78, 555, 594, 493, 185, 301, 233, 358,
        217,  77,  48, 158, 575, 522, 207, 566, 263, 512, 238, 447, 538,
        221,  67,  79, 188, 174, 109,  32, 509, 196, 421, 523,  92, 599,
        302, 506, 392, 531, 577, 339, 281, 399, 569

In [34]:
train_labels, test_labels = create_train_test(
    df=data[0], train_index=train_index, test_index=test_index, batch_size=64, cols=['target']
)

In [35]:
train_labels, test_labels

(<torch.utils.data.dataloader.DataLoader at 0x1d49ca7c610>,
 <torch.utils.data.dataloader.DataLoader at 0x1d49ca7c6d0>)

In [42]:
for data in train_labels:
    print(data)
data.shape

tensor([[1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [2],
        [1],
        [2],
        [1],
        [1],
        [1],
        [1],
        [2],
        [0],
        [2],
        [1],
        [2],
        [2],
        [1],
        [0],
        [2],
        [1],
        [2],
        [2],
        [0],
        [0],
        [2],
        [1],
        [2],
        [0],
        [0],
        [2],
        [2],
        [1],
        [2],
        [1],
        [0],
        [2],
        [1],
        [2],
        [0],
        [0],
        [2],
        [0],
        [1],
        [1],
        [1],
        [0],
        [2],
        [2],
        [1],
        [2],
        [1],
        [2],
        [0],
        [2]])
tensor([[1],
        [0],
        [1],
        [1],
        [2],
        [0],
        [0],
        [1],
        [2],
        [0],
        [1],
        [1],
        [1]

torch.Size([32, 1])

In [43]:
for data in test_labels:
    print(data)
data.shape

tensor([[0],
        [2],
        [0],
        [2],
        [1],
        [1],
        [1],
        [2],
        [1],
        [0],
        [0],
        [2],
        [1],
        [0],
        [1],
        [1],
        [2],
        [1],
        [1],
        [0],
        [2],
        [0],
        [0],
        [1],
        [0],
        [2],
        [0],
        [2],
        [0],
        [1],
        [0],
        [0],
        [2],
        [1],
        [0],
        [2],
        [2],
        [2],
        [2],
        [1],
        [1],
        [0],
        [2],
        [2],
        [1],
        [1],
        [2],
        [0],
        [2],
        [1],
        [0],
        [1],
        [2],
        [1],
        [2],
        [2],
        [1],
        [2],
        [2],
        [1],
        [2],
        [1],
        [1],
        [1]])
tensor([[0],
        [0],
        [1],
        [1],
        [2],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0]

torch.Size([56, 1])

In [61]:
import torch

batch_size, N, K = 256, 3, 3

x = torch.rand(batch_size, N, K) # [M, N, K]
y = torch.rand(batch_size, N, K) # [M, N, K]

output1 = torch.cat([x,y], dim=1) #[M, N+N, K]
output2 = torch.cat([x,y], dim=2) #[M, N, K+K]

In [62]:
output1

tensor([[[3.0350e-01, 1.4899e-01, 4.7743e-01],
         [9.4715e-01, 7.1139e-01, 8.0479e-01],
         [1.0336e-01, 3.2935e-01, 7.2725e-01],
         [7.5603e-01, 5.0983e-01, 6.5149e-01],
         [4.2878e-01, 7.5607e-01, 1.3795e-02],
         [4.1840e-02, 3.0173e-01, 5.3161e-01]],

        [[4.5237e-01, 8.8837e-01, 1.4768e-01],
         [2.0478e-02, 1.1304e-01, 2.4966e-01],
         [8.5099e-01, 3.2564e-01, 2.5473e-01],
         [9.9347e-01, 2.3294e-01, 1.6819e-01],
         [2.4605e-01, 5.0966e-01, 5.0287e-01],
         [1.6043e-01, 3.9312e-01, 4.2689e-01]],

        [[4.8051e-01, 7.7709e-01, 2.9097e-03],
         [3.8494e-01, 3.1591e-01, 1.7867e-01],
         [3.4842e-01, 4.5304e-01, 5.5520e-02],
         [9.7999e-02, 3.1484e-01, 1.0359e-02],
         [2.5829e-02, 8.5811e-01, 7.6582e-01],
         [8.6752e-01, 5.4237e-01, 9.4450e-04]],

        ...,

        [[9.1199e-01, 1.3941e-01, 7.0780e-01],
         [1.0556e-01, 8.6896e-01, 6.7687e-01],
         [8.0777e-02, 9.2459e-01, 8.0129

In [63]:
output2

tensor([[[3.0350e-01, 1.4899e-01, 4.7743e-01, 7.5603e-01, 5.0983e-01,
          6.5149e-01],
         [9.4715e-01, 7.1139e-01, 8.0479e-01, 4.2878e-01, 7.5607e-01,
          1.3795e-02],
         [1.0336e-01, 3.2935e-01, 7.2725e-01, 4.1840e-02, 3.0173e-01,
          5.3161e-01]],

        [[4.5237e-01, 8.8837e-01, 1.4768e-01, 9.9347e-01, 2.3294e-01,
          1.6819e-01],
         [2.0478e-02, 1.1304e-01, 2.4966e-01, 2.4605e-01, 5.0966e-01,
          5.0287e-01],
         [8.5099e-01, 3.2564e-01, 2.5473e-01, 1.6043e-01, 3.9312e-01,
          4.2689e-01]],

        [[4.8051e-01, 7.7709e-01, 2.9097e-03, 9.7999e-02, 3.1484e-01,
          1.0359e-02],
         [3.8494e-01, 3.1591e-01, 1.7867e-01, 2.5829e-02, 8.5811e-01,
          7.6582e-01],
         [3.4842e-01, 4.5304e-01, 5.5520e-02, 8.6752e-01, 5.4237e-01,
          9.4450e-04]],

        ...,

        [[9.1199e-01, 1.3941e-01, 7.0780e-01, 1.6094e-01, 1.6096e-01,
          7.4417e-02],
         [1.0556e-01, 8.6896e-01, 6.7687e-01, 8.72